In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Get Tag Data

## Create Client

In [2]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    tz="Europe/Brussels"
)

## Extract data from single tag
If we are planning to retrieve the latest data from tags, it is best to send index requests to make sure the data is up to date. This action happens automatically when using the TrendMiner UI.

In [3]:
level = client.tag.from_name("[CS]BA:LEVEL.1")
conc = client.tag.from_name("[CS]BA:CONC.1")
temp = client.tag.from_name("[CS]BA:TEMP.1")
phase = client.tag.from_name("[CS]BA:PHASE.1")

level.index()
conc.index()
temp.index()
phase.index()

### Example 1 - Interpolated data for fixed interval
Interpolated data is the default. The main advantage is that we get equidistant timestamps, which will be the same for every tag, as long as we use the same interval.

In [4]:
interval = client.interval("2023-01-01", "2023-01-03")
ser = level.get_data(
    interval, 
    form="interpolated", 
    resolution="1h"
)
ser.head()

ts
2023-01-01 00:00:00+01:00    28.735641
2023-01-01 01:00:00+01:00    10.856165
2023-01-01 02:00:00+01:00    14.464235
2023-01-01 03:00:00+01:00    40.634544
2023-01-01 04:00:00+01:00    24.326510
Name: [CS]BA:LEVEL.1, dtype: float64

### Example 2 - Index data for the last 8h
Retrieve the datapoints from the index data stored on the appliance. For every interval we request, we get up to 4 real points: the min, max, start and end datapoints, up to the appliance index resolution (requesting more intervals or a higher resolution will not return any additional datapoints, as those points are simply not stored in the index).

In [5]:
ser = conc.get_data(interval="8h", form="index", resolution="1m")
ser.head()

ts
2023-01-09 08:19:28.849000+01:00    43.696697
2023-01-09 08:20:11+01:00           44.697550
2023-01-09 08:21:11+01:00           42.726110
2023-01-09 08:21:41+01:00           44.948240
2023-01-09 08:22:11+01:00           43.347740
Name: [CS]BA:CONC.1, dtype: float64

### Example 3 - Chart data for a fixed interval
Requesting chart data is similar to retrieving index data, only here the request is passed on to the tag historian if we request a resolution higher than index resolution. This allows us to retrieve higher resolution data, but does take up historian resources.

In [6]:
interval = client.interval(
    "2023-01-01 10:00", 
    "2023-01-01 12:00"
)
ser = temp.get_data(interval, form="chart", resolution="10s")
ser.head()

ts
2023-01-01 10:00:00+01:00    22.739271
2023-01-01 10:00:41+01:00    21.663725
2023-01-01 10:01:11+01:00    18.954967
2023-01-01 10:01:41+01:00    18.744540
2023-01-01 10:02:11+01:00    16.549826
Name: [CS]BA:TEMP.1, dtype: float64

## Extract data from multiple tags
The way to get data from multiple tags at once is to first create a TrendHub view instance (that we needn't necessarly save to the appliance). We can give multiple intervals as layers too. The output will be a list of DataFrame instances.

In [7]:
intervals = [
    ("2023-01-01", "2023-01-02"), 
    ("2023-01-02", "2023-01-03")
]


thv = client.trend.view(
    entries=[level, conc, temp, phase],
    layers=intervals,
    live=False
)

Interpolated data makes the most sense if we get data from at trendhub view

In [8]:
df_list = thv.get_data(form="interpolated", resolution="1m")
df_list[0].head()

[CS]BA:LEVEL.1  [CS]BA:CONC.1  [CS]BA:TEMP.1  \
ts                                                                        
2023-01-01 00:00:00+01:00       28.735641      21.912792      19.872892   
2023-01-01 00:01:00+01:00       29.301245      21.421415      20.133770   
2023-01-01 00:02:00+01:00       30.277832      21.790558      20.684408   
2023-01-01 00:03:00+01:00       31.961744      21.475044      20.209772   
2023-01-01 00:04:00+01:00       32.365350      21.815203      20.324425   

                          [CS]BA:PHASE.1  
ts                                        
2023-01-01 00:00:00+01:00         Phase3  
2023-01-01 00:01:00+01:00         Phase3  
2023-01-01 00:02:00+01:00         Phase3  
2023-01-01 00:03:00+01:00         Phase3  
2023-01-01 00:04:00+01:00         Phase3

But we can get other types of data too, just be aware the timestamps will not necessarily match

In [9]:
df_list = thv.get_data(form="index", resolution="1m")
df_list[0].head()

[CS]BA:LEVEL.1  [CS]BA:CONC.1  [CS]BA:TEMP.1  \
ts                                                                        
2023-01-01 00:00:00+01:00       28.735641      21.912792      19.872892   
2023-01-01 00:00:11+01:00       29.031664      22.112310      19.813826   
2023-01-01 00:00:41+01:00       28.712177      20.814423      20.637138   
2023-01-01 00:01:11+01:00       29.642284      21.772831      19.842346   
2023-01-01 00:01:41+01:00       29.744593      22.076654            NaN   

                          [CS]BA:PHASE.1  
ts                                        
2023-01-01 00:00:00+01:00         Phase3  
2023-01-01 00:00:11+01:00            NaN  
2023-01-01 00:00:41+01:00            NaN  
2023-01-01 00:01:11+01:00            NaN  
2023-01-01 00:01:41+01:00            NaN

We can fill in the missing values through interpolation (linear or stepped, depending on the tag type) by setting :attr:`fill=True`.

In [10]:
df_list = thv.get_data(form="index", resolution="1m", fill=True)
df_list[0].head()

[CS]BA:LEVEL.1  [CS]BA:CONC.1  [CS]BA:TEMP.1  \
ts                                                                        
2023-01-01 00:00:00+01:00       28.735641      21.912792      19.872892   
2023-01-01 00:00:11+01:00       29.031664      22.112310      19.813826   
2023-01-01 00:00:41+01:00       28.712177      20.814423      20.637138   
2023-01-01 00:01:11+01:00       29.642284      21.772831      19.842346   
2023-01-01 00:01:41+01:00       29.744593      22.076654      20.357895   

                          [CS]BA:PHASE.1  
ts                                        
2023-01-01 00:00:00+01:00         Phase3  
2023-01-01 00:00:11+01:00         Phase3  
2023-01-01 00:00:41+01:00         Phase3  
2023-01-01 00:01:11+01:00         Phase3  
2023-01-01 00:01:41+01:00         Phase3